#### Data cleaning and EDA

In [1]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
class DataCleaning:
    def __init__(self, data):
        self.data = data
    
    
    def null_values(self):
        '''Función que toma un dataframe y calcula el porcentaje de valores nulos para las columnas'''
        null_val = self.data.isnull().sum()
        null_val_percent = 100 * self.data.isnull().sum() / len(self.data)
        null_val_table = pd.concat([null_val, null_val_percent], axis=1,keys=['Valores Nulos','% del total de valores'])   

        if null_val_table[null_val_table.iloc[:,1] != 0].shape[0]>0:

            null_val_table = null_val_table[
                null_val_table.iloc[:,1] != 0].sort_values(
            '% del total de valores', ascending=False).round(1)
            print ("Hay " + str(null_val_table.shape[0]) +
                  " variables que tienen valores nulos")
            return null_val_table
        else:
            print ("No hay variables que tengan valores nulos")
        
        
    def data_types(self):
        '''Función que toma un dataframe y regresa el tipo de dato que contienen las columnas'''
        dt_types = self.data.dtypes
        return dt_types
    
    def unique_values(self):
        '''Función que toma un dataframe y regresa los valores únicos que contienen las columnas'''
        unique_values = {}
        for column in self.data.columns:
            unique_values[column] = self.data[column].nunique()
        return unique_values
    
    def cross_table(self,columns,y_column):
        cross_tab = pd.crosstab(index=self.data[self.y_column], columns=self.data[list(columns)])
        return cross_tab

In [ ]:
class Outlier_analysis:
    def __init__(self, data):
        self.data = data
        
    def descr_stats(self,column):
        column_data = self.data[column]
        Q1 = np.percentile(column_data,25)
        Q3 = np.percentile(column_data,75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = column_data[(column_data < lower_bound) | (column_data > lower_bound)]
        return outliers
    
    def plot_boxplot(self, column):
        sns.boxplot(x=self.data[column])
        plt.title(f'Boxplot de {column}')
        plt.show()
        
        
    def correct_outliers(self, column, method='median'):
        column_data = self.data[column]
        Q1 = np.percentile(column_data,25)
        Q3 = np.percentile(column_data,75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR   
        
        if method == 'median':
            median = column_data.median()
            self.data.loc[column_data < lower_bound, column] = median
            self.data.loc[column_data > upper_bound, column] = median
        elif method == 'mean':
            mean = column_data.mean()
            self.data.loc[column_data < lower_bound, column] = mean
            self.data.loc[column_data > upper_bound, column] = mean
        else:
            print("Método inválido. Eliga 'median' or 'mean'.")

In [ ]:
class eda_plots:
    def __init__(self,data):
        self.data = data
            
    def plot_histogram(self,columns):
        '''Función que grafica histograma'''
        for col in columns:
            sns.histplot(self.data[col],kde=True)
            plt.title(f'Histograma de {col}')
            plt.xlabel(col)
            plt.ylabel('Frecuencia')
            plt.show()
            
    def plot_bar(self, x_column, y_column):
        '''Función que identifica columnas categóricas y hace un barplots'''
        sns.barplot(x=self.data[x_column], y=self.data[y_column])
        plt.title(f'Barplot de {y_column} con {x_column}')
        plt.xlabel(x_column)
        plt.ylabel(y_column)
        plt.show()

        
    def plot_boxplot(self,columns):
        '''Función que identifica columnas numéricas y hace un boxplot'''
        for col in columns:   
            sns.boxplot(x=self.data[col])
            plt.title(f'Boxplot de {col}')
            plt.xlabel(col)
            plt.show()
    
    def plot_scatterplot(self,x_column,y_column):
        if self.data[column].dtype != 'object':
            plt.figure(figsize=(8,6))
            sns.scatterplot(x=self.data.index, y=self.data[column])
            plt.title(f'Scatterplot de {y_column} vs {x_column}')
            plt.xlabel(x_column)
            plt.ylabel(y_column)
            plt.show()
    
    def heatmap_numeric_w_target(self, y_column):
        '''Función que toma la variable dependiente y dependientes y regresa un mapa de calor con correlaciones'''
        plt.figure(figsize=(8, 10))
        g = sns.heatmap(self.data.corr()[[y_column]].sort_values(by=y_column), 
                        annot=True, 
                        cmap='coolwarm', 
                        vmin=-1,
                        vmax=1) 
        return g
    
    
    def numeric_target(self):
        numeric_columns = self.data.select_dtypes(include=['float']).columns
    
        for i, col in numeric_columns:
            plt.figure(figsize=(16, 25))
            plt.subplot(len(numeric_columns), 4, i*2+1)
            plt.subplots_adjust(hspace =.25, wspace=.3)
            plt.grid(True)
            plt.title(col)
            sns.kdeplot(self.data.loc[self.data["isFraud"]==0, col], color = "pink", shade=True, kernel='gau', cut=0)
            sns.kdeplot(self.data.loc[self.data["isFraud"]==1, col], color = "skyblue", shade=True, kernel='gau', cut=0)
            plt.subplot(6, 4, i*2+2) 
            sns.boxplot(y = col, data = self.data, x="y", palette = ["pink", "skyblue"])
            
            
    def categor_target(self):
        non_numeric_cols = self.data.select_dtypes(include=['object']).columns
                    
        plt.figure(figsize=(13, 9))
        for i, var in enumerate(non_numeric_cols):
            plt.subplot(2, 1, i+1)
            plt.title(var, fontsize=14)
            plt.xlabel(var, fontsize=12)
            plt.ylabel("Count", fontsize=12)
            plt.subplots_adjust(hspace = 0.4, wspace = 0.3)
            sns.countplot(data= self.data, x = var, hue=self.data["isFraud"], palette = ['pink', "skyblue"])